### Q-3. Imagine you have a dataset where you have different categories of data, Now you need to find the most similar data to the given data by using any 4 different similarity algorithms. Now you have to build a model which can find the most similar data to the given data.


In [29]:
import nltk
from nltk.corpus import stopwords # To handle stopwords dictionary
from nltk.stem import WordNetLemmatizer # lemmatization
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string #to handle punctuations and special characters
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report, adjusted_rand_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer # import CountVectorizer to create DTM
from sklearn.feature_extraction.text import TfidfTransformer # import to convert DTM to TF-IDF
from sklearn.cluster import KMeans
from kneed import KneeLocator
from sklearn.metrics import silhouette_score

In [30]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aneesaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/aneesaro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aneesaro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aneesaro/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [31]:
df = pd.read_json("News_Category_Dataset_v3.json", lines=True)
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [32]:
df.authors = df["authors"].apply(lambda x: x.split(",")[0])

In [33]:
df["authors"].value_counts()

                                          37418
Lee Moran                                  2955
Ron Dicker                                 2219
Ed Mazza                                   1591
Reuters                                    1577
                                          ...  
EMMANUEL JARRY AND ROBERT-JAN BARTUNEK        1
JIM SALTER & JIM SUHR                         1
BRIAN GROSH                                   1
Collin Binkley and Errin Haines Whack         1
John Giacobbi                                 1
Name: authors, Length: 25063, dtype: int64

In [34]:
lemmatizer = WordNetLemmatizer()

In [35]:
nlp_cols = ["headline", "short_description"]

nlp_pipeline = Pipeline([
    ('count_vectorizer', CountVectorizer()),
    ('tfidf_transformer', TfidfTransformer())
])

In [36]:
# Define a function to map the part of speech tags from NLTK to WordNet
def get_wordnet_pos(word):
    """Map POS tag to first character used by WordNet lemmatizer"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def message_text_process(mess):
    mess = remove_word_with_digit_and_non_ascii(mess)
    # remove punctuation from string
    no_punctuation = [char for char in mess if char not in string.punctuation] 
    no_punctuation = ''.join(no_punctuation) # joining all the characters without punctuation
    # remove stop words
    words = [word.lower() for word in no_punctuation.split() if word not in stopwords.words('english')]
    words = " ".join(words)
    # lemmatize the text
    tokens = nltk.word_tokenize(words)
    lemmas = []
    for token in tokens:
        lemma = lemmatizer.lemmatize(token, pos=get_wordnet_pos(token))
        lemmas.append(lemma)
    return lemmas

def remove_word_with_digit_and_non_ascii(sentence):
    final_sentence = ""
    sentence = ''.join(c for c in sentence if ord(c) < 128)
    for word in sentence:
        if not any(char.isdigit() for char in word):
            final_sentence += word
    return final_sentence


In [37]:
X = df.drop(["date", "link"], axis=1)

In [38]:
tfidf_transformer = TfidfTransformer()

count_vectorizer = CountVectorizer(analyzer=message_text_process)
X_train_bagwords = count_vectorizer.fit_transform(X["headline"])
X_train_tfidf = tfidf_transformer.fit_transform(X_train_bagwords)

count_vectorizer = CountVectorizer(analyzer=message_text_process)
X_train_bagwords_2 = count_vectorizer.fit_transform(X["short_description"])
X_train_tfidf_2 = tfidf_transformer.fit_transform(X_train_bagwords_2)

In [39]:
print(X_train_tfidf.shape)
print(X_train_tfidf_2.shape)

(209527, 55202)
(209527, 77429)


In [40]:
# Combine the sparse matrices
train_arr = sparse.hstack((X_train_tfidf_2, X_train_tfidf))

print(train_arr)

  (0, 76079)	0.1761488231929779
  (0, 75102)	0.21493621669984492
  (0, 71033)	0.15590526641403696
  (0, 59738)	0.13914186762046307
  (0, 53657)	0.3124966409027681
  (0, 49261)	0.23519451605803626
  (0, 46457)	0.1530436814592696
  (0, 43424)	0.21760227246131403
  (0, 41689)	0.2848370302496648
  (0, 29709)	0.19954409781496518
  (0, 22675)	0.22923295575386676
  (0, 22215)	0.24965763800549143
  (0, 19784)	0.24027084079289215
  (0, 18899)	0.37020362588712913
  (0, 16998)	0.2648519486667227
  (0, 8085)	0.39320395952163173
  (0, 129252)	0.19958875504935653
  (0, 122735)	0.4332176863429729
  (0, 119424)	0.3004921222703465
  (0, 113109)	0.20360321673985385
  (0, 112432)	0.5053056399828995
  (0, 108785)	0.246813544200962
  (0, 95511)	0.13526587305463647
  (0, 88257)	0.30398160318286566
  (0, 83302)	0.4070425623309862
  :	:
  (209525, 80083)	0.2712330225019758
  (209525, 78471)	0.4788785832542771
  (209526, 70709)	0.35378991830181156
  (209526, 69520)	0.3251863314405932
  (209526, 68199)	0.089755

In [42]:
# Encode the categories
encoder = LabelEncoder()
X["category"] = encoder.fit_transform(X["category"])
X["authors"] = encoder.fit_transform(X["authors"])

train_arr = sparse.hstack((train_arr, np.array(X["authors"])[:, None], np.array(X["category"])[:, None]))

In [43]:
wcss=[]
for k in range(2,40):
    kmean=KMeans(n_clusters=k,init="k-means++")
    kmean.fit(train_arr)
    wcss.append(kmean.inertia_)

k = KneeLocator(range(2,40), wcss, curve='convex', direction='decreasing')
optimal_clusters = k.elbow

print(f"Optimal_Clusters: {optimal_clusters}")

/Users/aneesaro/miniforge3/envs/pyc/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Optimal_Clusters: 7


In [44]:
# Initialize and fit the K-Means model
kmeans = KMeans(n_clusters=optimal_clusters, init="k-means++", random_state=42)  # Specify the number of clusters
kmeans.fit(train_arr)

KMeans(n_clusters=7, random_state=42)

In [45]:
silhouette_avg = silhouette_score(train_arr, kmeans.labels_)
print("The training silhouette score is:", silhouette_avg)

The training silhouette score is: 0.6189240054929624


In [47]:
df["clusters"] = kmeans.labels_